In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.firefox.service import Service
import PRIVATE
import random

def sleep_time(min_sec, max_sec):
  time.sleep(random.uniform(min_sec,max_sec))
  driver.implicitly_wait(random.uniform(min_sec,max_sec))
  
# driver setting
driver_path = "./gecko/geckodriver"
options = webdriver.FirefoxOptions()
# options.add_argument('--headless')
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")

service = Service(executable_path=driver_path)
driver = webdriver.Firefox(service=service, options=options)

# id & pw
everytime_id = PRIVATE.EVERYTIME_ID
everytime_pw = PRIVATE.EVERYTIME_PW

##############

driver.get("https://account.everytime.kr/login")
sleep_time(1,2)

driver.find_element(By.NAME, 'id').send_keys(everytime_id)
driver.find_element(By.NAME, 'password').send_keys(everytime_pw)
sleep_time(1,2)

# driver.find_element(By.XPATH,'//*[@class="submit"]/input').click()
# driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()
driver.find_element(By.CSS_SELECTOR, '.container > form:nth-child(2) > input:nth-child(2)').click()
sleep_time(1,2)

